In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Activation, Embedding, LSTM
from sklearn.model_selection import train_test_split
import scipy.signal
import numpy as np
import pandas
import plotly as py
import plotly.graph_objs as go
from plotly import tools
from matplotlib.pyplot import figure
import random
from heapq import nlargest
import math

In [ ]:
"""
Input: Driving Inertial Signals 
Preprocess: Scale and Smoothing the signals 
Run 3 classifiers to detect Acc/Brake, RT/LT, UT 
Output: List of events with their intensities to analyse the driver's behavior

"""
### Driver 1
# address = './MyData/22.4.2020.part2.csv'
address = './MyData/22.4.2020.part3.csv'
# address = './MyData/5.5.2020/2.5.2020 200505 16_40_37.csv'
# address = './MyData/5.5.2020/2.5.2020 200505 17_12_00.csv'

###Driver 2
# address = './MyData/7.5.2020/2.5.2020 200507 19_44_25.csv'
# address = './MyData/7.5.2020/2.5.2020 200507 19_58_41.csv'

###Driver 3
# address = './MyData/7.5.2020/2.5.2020 200508 00_40_14.csv'
# address = './MyData/7.5.2020/2.5.2020 200508 00_19_22.csv'

## Test Data
# address = './data/Learning/Aggressive_from_jair/21.csv'
# address = './data/Learning/Aggressive_from_jair/round5_labeled.csv'

# address = './MyData/old_one.csv'
# address = './MyData/2.5.2020/round6.csv'

data = pandas.read_csv(address,  usecols=[8,23,24,25,15,16,17,29,30,31])
# data = pandas.read_csv(address)

X_turn = data
X_acc = data[['motionUserAccelerationX(G)','motionUserAccelerationY(G)','motionUserAccelerationZ(G)']]

In [ ]:
model_acc = load_model_acc()
model_turn = load_model_turn()
model_uturn = load_model_uturn()

In [ ]:
X_acc = np.array(X_acc)
X_acc = SmoothData(X_acc)
X_turn = np.array(X_turn)
X_turn = SmoothData(X_turn)

In [ ]:
section = len(X_acc) - (len(X_acc) % 200)
### Run the model based on input shape 
results_acc = model_acc.predict(np.array(X_acc[:section]).reshape(int(section/200),200,3))
results_acc = results_acc.reshape(section,3)

results_turn = model_turn.predict(np.array(X_turn[:section]).reshape(int(section/200),200,10))
results_turn = results_turn.reshape(section,3)

results_uturn = model_uturn.predict(np.array(X_turn[:section]).reshape(int(section/200),200,10))
results_uturn = results_uturn.reshape(section,2)


In [ ]:
### giving data to the create_turn_event

events_turn = create_turn_event(X_turn[:section], one_hot_decode(results_turn))
events_turn = assign_intensity_turn(events_turn, X_turn[:section])
events_turn, deletion_index = merge_turn(events_turn)
events_turn.remove(('',math.inf,math.inf,math.inf))
del events_turn[0:max(deletion_index[-1])+1]

In [ ]:
### giving acceleration data to the create_acc_event

events_acc = create_acc_event(X_acc[:section], one_hot_decode(results_acc))
events_acc = assign_intensity_acc(events_acc, X_acc[:section])
events_acc, deletion_index = merge_acc(events_acc)
events_acc.remove(('',math.inf,math.inf,math.inf))
del events_acc[0:max(deletion_index[-1])+1]

In [ ]:
### giving acceleration data to the create_uturn_event

events_uturn = create_uturn_event(X_turn[:section], one_hot_uturn_decode(results_uturn))
events_uturn = assign_intensity_turn(events_uturn, X_turn[:section])
events_uturn, deletion_index = merge_uturn(events_uturn)
events_uturn.remove(('',math.inf,math.inf,math.inf))
del events_uturn[0:max(deletion_index[-1])+1]

In [ ]:
"""
Merge all events together
"""
all_events = []
all_events += events_acc
all_events += events_turn
all_events += events_uturn

all_events.sort(key=lambda tup: tup[1])


In [ ]:
"""
Calculate the score of driving
E = (k1*Low + k2*Medium + k3*High) / duration
"""

def score_calc(data):
    score = 0
    init = data[0][1]
    end = data[-1][2]
    freq = 50
    duration = (end - init) /freq
    print("duration:",(end-init)/freq)
    for i in data:
        if(i[4]==0):
            score += 1
        elif(i[4]==1):
            score += 2
        elif(i[4]==2):
            score += 4
    print("score", score)
    return score/duration


In [ ]:
def CDF (data, x):
    count = 0
    for i in data:
        if(x >= i):
            count +=1
    return count/len(data)

In [ ]:
scores = []
for i in all_scores:
    scores.append(1- CDF(data, i))
print("Score per time unit:", scores)
print("Mean Score:", np.mean(scores))

In [ ]:
"""
Extract information from events 
"""

acc_df = pandas.DataFrame(events_acc)
turn_df = pandas.DataFrame(events_turn)
uturn_df = pandas.DataFrame(events_uturn)

if(len(acc_df)):
    acc_dist = np.array(acc_df.groupby([4]).size())
else:
    acc_dist = np.array([0,0,0])

if(len(turn_df)):
    turn_dist = np.array(turn_df.groupby([4]).size())
else :
    turn_dist = np.array([0,0,0])
    
if(len(uturn_df)):
    uturn_dist = np.array(uturn_df.groupby([4]).size())
else:
    uturn_dist = np.array([0,0,0])

# Utility functions

In [ ]:
def one_hot_decode(results):
    result_class = []
    for i in results:
        if(np.argmax(i)==0):
            result_class.append(0)
        elif(np.argmax(i)==1):
            result_class.append(1)
        elif(np.argmax(i)==2):
            result_class.append(2)
    return result_class

def one_hot_uturn_decode(results):
    result_class = []
    for i in results:
        if(np.argmax(i)==0):
            result_class.append(0)
        else:
            result_class.append(1)
    return result_class

In [ ]:
def SmoothData(data):
    size = data.shape[0] 
    dim = data.shape[1]
    
    smooth_data = np.zeros((size,dim))
    
    smooth_data[0] = data[0]
    smooth_data[1] = data[1]
    smooth_data[size-1] = data[size-1]
    smooth_data[size-2] = data[size-2]
    
    for i in range(2, size-2):
        smooth_data[i] = np.average([data[i], data[i-1], data[i+1]], axis=0)
    return smooth_data

In [ ]:
def SmoothData(data):
    size = data.shape[0] 
    dim = data.shape[1]
    
    smooth_data = np.zeros((size,dim))
    
    smooth_data[0] = data[0]
    smooth_data[1] = data[1]
    smooth_data[size-1] = data[size-1]
    smooth_data[size-2] = data[size-2]
    
    for i in range(2, size-2):
        smooth_data[i] = np.average([data[i], data[i-1], data[i+1]], axis=0)
    return smooth_data

In [ ]:
"""
Utitlity Functions
"""
"""
Computing the energy (on y-axis)

First, scale the accelerometer signal to the 9.8 
Second, iterate list of events, calculate energy of the y-axis 
"""

def Energy(data):
    data = [abs(number) for number in data]
    return np.sum(np.power(data,2))

    
"""
Scaling turn signal to m/s2
"""
def Scale(data):
    scaled_data = data.copy()
    for i in range(len(scaled_data)):
        scaled_data[i] = scaled_data[i] * 9.8
    return scaled_data



In [ ]:
"""
Creating list of events and 
removing events with energy < 20 and duration lower than 50

"""
def create_acc_event(data, result_class):
    
    brks = 0
    accs = 0
    num_events = 0
    events = []
    freq = 100     
    scaled_data = Scale([data[i][0] for i in range(len(data))]) # ax
    for i in range(len(result_class)):
        if(result_class[i]==1):
            brks += 1
        elif(result_class[i]==2):
            accs += 1
        else:
            if(brks > 50 ):
                signal = [abs ((scaled_data)[i]) for i in range(i-brks, i-1)]
                energy = Energy(signal)
                if(energy > 10):
                    events.append(("brake", i-brks, i-1, brks/freq))
                    num_events += 1
                brks = 0
            elif(accs > 50):
                signal = [abs ((scaled_data)[i]) for i in range(i-accs, i-1)]
                energy = Energy(signal)
                if(energy > 10):
                    events.append(("acc", i-accs, i-1, accs/freq))
                    num_events += 1
                accs = 0
            
    return events

In [ ]:

def create_turn_event(data, result_class):
    rt = 0
    lt = 0
    num_events = 0
    num_rt = 0
    num_lt = 0
    events = []
    freq = 100 
    scaled_data = Scale([data[i][8] for i in range(len(data))]) # ay

    for i in range(len(result_class)):
        if(result_class[i]==1):
            rt += 1
        elif(result_class[i]==2):
            lt += 1
        else:
            if(rt > 100 ):
                signal = [abs ((scaled_data)[i]) for i in range(i-rt, i-1)]
                energy = Energy(signal)
                if(energy > 10):
                    events.append(("rt", i-rt, i-1, rt/freq))
                    num_events += 1
                rt = 0
            elif(lt > 100):
                signal = [abs ((scaled_data)[i]) for i in range(i-lt, i-1)]
                energy = Energy(signal)
                if(energy > 10):
                    events.append(("lt", i-lt, i-1, lt/freq))
                    num_events += 1
                lt = 0
    return events

In [ ]:
def create_uturn_event(data, result_class):
    ut = 0
    num_events = 0
    events = []
    freq = 100 
    scaled_data = Scale([data[i][8] for i in range(len(data))]) # ay

    for i in range(len(result_class)):
        if(result_class[i]==1):
            ut += 1
        else:
            if(ut > 100):
                signal = [abs ((scaled_data)[i]) for i in range(i-ut, i-1)]
                energy = Energy(signal)
                if(energy > 10):
                    events.append(("ut", i-ut, i-1, ut/freq))
#                     print(num_events, "- ut event," ,"duration:", ut/freq, " tsteps", "init:", i-ut ,"end:", i-1)
                    num_events += 1
                ut = 0
    return events

In [ ]:
def ComputeIntensity(signal):
    max_values = np.mean(nlargest(3, signal))
    if(max_values > 0.08 and max_values < 0.2):
        return 0
    elif(max_values > 0.2 and max_values< 0.4):
        return 1
    elif(max_values > 0.4):
        return 2  

In [ ]:
"""
Assign intensity to each event 
0, 1, 2 --> Low, Medium, High
thresholds:
    0.98 = 0.1G
    1.96 = 0.2G
    3.92 = 0.4G
"""    
def assign_intensity_acc(events, data):
    i = 0
    for e in events:
        event_type, init, end, duration = e
        signal = [abs(data)[i][0] for i in range(init, end)]
        events[i] += (ComputeIntensity(signal),)
        i +=1
    return events

def assign_intensity_turn(events, data):
    i = 0
    for e in events:
        event_type, init, end, duration = e
        signal = [abs(data)[i][8] for i in range(init, end)]
        events[i] += (ComputeIntensity(signal),)
        i +=1
    return events



In [ ]:
"""
merging neighbour events
Input: events
Output: (merge acc neighbour events, deletion index)
"""
def merge_acc(events):    
    duration = 0
    merge_index = 0
    deletion_index = []
    events.append(('',math.inf,math.inf,math.inf))

    for i in range(0,len(events)-1):
        if(abs((events[i][2])-(events[i+1][1]))<11 and events[i][0]==events[i+1][0]):
            duration +=1
        else:
            init = events[i-duration][1]
            end = events[i][2]-1
            events.append((events[i-duration][0], init, end, end-init, events[i-duration][4]))
            deletion_index.append(np.arange(i-duration,i+1))
            duration = 0
    return (events, deletion_index)

def merge_turn(events):    
    duration = 0
    merge_index = 0
    deletion_index = []
    events.append(('',math.inf,math.inf,math.inf))

    for i in range(0,len(events)-1):
        if(abs((events[i][2])-(events[i+1][1]))<11 and events[i][0]==events[i+1][0]):
            duration +=1
        else:
            init = events[i-duration][1]
            end = events[i][2]-1
            events.append((events[i-duration][0], init, end, end-init, events[i-duration][4]))
            deletion_index.append(np.arange(i-duration,i+1))
            duration = 0
    return (events, deletion_index)


def merge_uturn(events):
    duration = 0
    merge_index = 0
    deletion_index = []
    events.append(('',math.inf,math.inf,math.inf))

    for i in range(0,len(events)-1):
        if(abs((events[i][2])-(events[i+1][1]))<11 and events[i][0]==events[i+1][0]):
            duration +=1
        else:
            init = events[i-duration][1]
            end = events[i][2]-1
            events.append((events[i-duration][0], init, end, end-init, events[i-duration][4]))
            deletion_index.append(np.arange(i-duration,i+1))
            duration = 0
    return (events, deletion_index)

In [ ]:
# load json and create model
"""
Output: Acc/Brake detection model
"""
from keras.models import model_from_json
import os

def load_model_acc():
    json_file = open('./Learning_model/acc,brake/v2/model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("./Learning_model/acc,brake/v2/model.h5")
    print("Loaded model_acc from disk")
    model = loaded_model
    return model


def load_model_turn():
    json_file = open('./Learning_model/Turning/rt_lt_cv/v2/model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("./Learning_model/Turning/rt_lt_cv/v2/model.h5")
    print("Loaded model_turn from disk")
    model = loaded_model
    return model


def load_model_uturn():
    json_file = open('./Learning_model/Turning/ut_cv/model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("./Learning_model/Turning/ut_cv/model.h5")
    print("Loaded model_uturn from disk")
    model = loaded_model
    return model